In [ ]:
# import netCDF4
import pandas as pd
import datetime as dt
import numpy as np
import csv
import calendar

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from environmentVariables import environment_variables

In [ ]:
userTemp = os.getlogin()

if userTemp == 'Eshta':
    user = 'eshta'
user_key=environment_variables[user]
pd.set_option('display.precision', 12)

# INPUT

In [ ]:
# read files

#we must automate all the files here

#the variable decides which file to select as data
#load in the csv with the selected options
selections=pd.read_csv('AutomatedSteps/optionSelection.csv', header=[0])
#now we allocate all variables (rlat,rlong,variable)
variable = ""
rlat = 0
rlong = 0
for index, row in selections.iterrows():
    if selections.at[index,'option'] == 'rlat':
        rlat = selections.at[index,'selection']
    if selections.at[index,'option'] == 'rlong':
        rlong = selections.at[index,'selection']
    if selections.at[index,'option'] == 'variable':
        variable = selections.at[index,'selection']
    if selections.at[index,'option'] == 'startDate':
        startDate = selections.at[index,'selection']
    if selections.at[index,'option'] == 'endDate':
        endDate = selections.at[index,'selection']
    if selections.at[index,'option'] == 'currYear':
        currYear = selections.at[index,'selection']
        
#now that selections are set we continue
#decide file based on date
rlat = int(rlat)
rlong = int(rlong)
#now based on how many files exist with variable name and date
if currYear == '1986':
    print("1986 here")
    dtStr = "19860101-19901231"
    dtOutStr = "1986-1990"
elif currYear == '1991':
    dtStr = "19910101-19951231"
    dtOutStr = "1991-1995"
elif currYear == '1996':
    dtStr = "19960101-20001231"
    dtOutStr = "1996-2000"
elif currYear == '2001':
    dtStr = "20010101-20051231"
    dtOutStr = "2001-2005"

    

    
data=pd.read_csv(user_key['path_in']+ variable + '/' + variable + '_' + dtStr + '.csv', header=None)
data_time = pd.read_csv(user_key['path_in']+ variable + '/' +'time_' + dtStr + '.csv', header=None)


# Calculate num of leap years

In [ ]:
# Input list initialization
input_list = [1950, 1951, 1952, 1953, 1954, 1955,
         1956, 1957, 1958, 1959, 1960,
         1961, 1962, 1963, 1964, 1965, 
         1966, 1967, 1968, 1969, 1970, 
         1971, 1972, 1973, 1974, 1975, 
         1976, 1977, 1978, 1979, 1980, 
         1981, 1982, 1983, 1984, 1985
        ,1986, 1987, 1988, 1989, 1990
        ,1991, 1992, 1993, 1994, 1995
        ,1996, 1997, 1998, 1999, 2000
        ]

In [ ]:
def findLeapYear(input_list):
    ans = 0
    for elem in input_list:
        if calendar.isleap(int(elem)):
            ans = ans + 1
    return ans

numleap = findLeapYear(input_list)
print(numleap)

# Extract based on rlat and rlon

In [ ]:
rlon_x = rlong # adjusted from 212 since headers start at 0
rlat_y = rlat  # adjusted from 120 since index starts at 0
counter = 0

list_of_vars = []

 
while (rlat_y+counter) < len(data)-1:
    var = data.iloc[rlat_y+counter, rlon_x]
    list_of_vars.append(var)
        
    counter+=260 

len(list_of_vars) #should always be 1825

In [ ]:
#read data_time and transpose into column and append variable and date info to new df
data_all = data_time.transpose()
data_all.insert(1, 'Variable', list_of_vars)
data_all = data_all.rename(columns={0: 'Date'})

In [ ]:
#data_all

# Change time units - new

In [ ]:
startdate = "12/01/1949"
temp = pd.to_datetime(startdate) + pd.DateOffset(days= 13171.5 +numleap)

In [ ]:
#print(temp)

In [ ]:
# change time units from 'days since 12/01/1949' to actual date
startdate = "12/01/1949"
row_iterator = data_all.iterrows()
counter = 1

for i, row in row_iterator:
    if counter != len(data_all)+1:
        #temp = pd.to_datetime(startdate) + pd.DateOffset(days=data_all.at[i, 'Date']+9)
        #if temp.month != 2 and temp.day != 29:
            #data_all.at[i, 'Date_new'] = pd.to_datetime(temp)
        data_all.at[i, 'Date_new'] = pd.to_datetime(startdate) + \
                                     pd.DateOffset(days=data_all.at[i, 'Date']+numleap)
            #data_all.at[i, 'Date_new2'] = pd.to_datetime(data_all.at[i, 'Date_new']) + pd.DateOffset(days=9)
            
    counter += 1

data_all['Date_new'] = pd.to_datetime(data_all['Date_new']).dt.date

In [ ]:
# flag leap years

row_iterator = data_all.iterrows()
counter = 1

for i, row in row_iterator:
    if counter != len(data_all)+1:
        if data_all.at[i, 'Date_new'].month == 2 and data_all.at[i, 'Date_new'].day == 29:
             data_all.at[i, 'flag'] = 'flag'

    counter += 1

In [ ]:
# if flag, change flagged date to +1

row_iterator = data_all.iterrows()
counter2 = 1

for i, row in row_iterator:
    if counter2 != len(data_all)+1:
        if data_all.at[i, 'flag'] == 'flag':
            data_all.at[i, 'Date_2'] = data_all.at[i+1, 'Date_new']
    counter2+=1   

In [ ]:
# copy over date from before if Date_2 is null 
# otherwise keep the value in Date_2

row_iterator = data_all.iterrows()
counter3 = 1
numberofflags = 0

for i, row in row_iterator:
    if counter3 != len(data_all)+1:
        if data_all.at[i, 'flag'] == 'flag':
            numberofflags+=1
        temp = data_all.at[i, 'Date_new'] + pd.DateOffset(days=numberofflags)
        if (temp.month == 2 and temp.day == 29):
            data_all.at[i, 'Date_2'] = data_all.at[i, 'Date_new'] + pd.DateOffset(days=numberofflags+1)
        else:
            data_all.at[i, 'Date_2'] = data_all.at[i, 'Date_new'] + pd.DateOffset(days=numberofflags)


    counter3+=1

data_all['Date_2'] = pd.to_datetime(data_all['Date_2']).dt.date

In [ ]:
#data_all

# OUTPUT

In [ ]:
# delete unnecessary columns
del data_all['Date']
del data_all['Date_new']
del data_all['flag']

# rename column
data_all = data_all.rename(columns={'Date_2': 'Date'})

In [ ]:
# reorder column
data_all = data_all[['Date', 'Variable']]

In [ ]:
data_all.to_csv('AutomatedSteps/' + variable +'_'+dtOutStr +'.csv', index=None)

# LEGACY CODE

# Change time units

In [ ]:
# # change time units from 'days since 12/01/1949' to actual date
# startdate = "12/01/1949"
# row_iterator = data_time.iterrows()
# counter = 1
# data_ActualDate = df = pd.DataFrame(columns = ['Date_new'])
# for i, row in row_iterator:
#     if counter != len(data_time) + 1:
#         temp = pd.to_datetime(startdate) + pd.DateOffset(days=data_time.at[i, 'Date'] + 9)
#         if (temp.month == 2 and temp.day == 29):
#             random = 1
#         else:
#             #data_all = data_time 
#             #data_all.at[i, 'Variable'] = pd.to_datetime(startdate) + pd.DateOffset(days=data_all.at[i, 'Date'] + 9)            
#             data_all.at[i, 'Date_new'] = pd.to_datetime(startdate) + pd.DateOffset(days=data_all.at[i, 'Date'] + 9)      
#          #   data = pd.to_datetime(startdate) + pd.DateOffset(days=data_time.at[i, 'Date'] + 9)
            
#         #    data_ActualDate = data_ActualDate.append({'Date_new' : data}, ignore_index=True)
            
#             #data_all.at[i, 'Date_new2'] = pd.to_datetime(data_all.at[i, 'Date_new']) + pd.DateOffset(days=9)
            
#     counter += 1

# data_ActualDate['Date_new'] = pd.to_datetime(data_ActualDate['Date_new']).dt.date
# #data_all.dropna(subset = ['Date_new'], inplace=True)
# #print(data_ActualDate.to_string())
# #data_ActualDate.to_csv(user_key['csv_loc_raw']+'/pr/pr_output_1986-1990-test3.csv', index=None)

In [ ]:
# flag leap years

# row_iterator = data_all.iterrows()
# counter = 1

# for i, row in row_iterator:
#     if counter != len(data_all)-1:
#         if data_all.at[i, 'Date_new'].month == 2 and data_all.at[i, 'Date_new'].day == 29:
#              data_all.at[i, 'flag'] = 'flag'

#     counter += 1

In [ ]:
# change leap year date to blank

#row_iterator = data_all.iterrows()
#counter2 = 1

#for i, row in row_iterator:
#    if counter2 != len(data_all)-1:
#        if data_all.at[i, 'flag'] == 'flag':
#            data_all.at[i, 'Date_2'] = data_all.at[i+1, 'Date_new']
#    counter2+=1   

# Check and solve nulls

In [ ]:
# forward fill leap year with next date and adjust following dates

#row_iterator = data_all.iterrows()
#counter3 = 1

#for i, row in row_iterator:
#    if counter3 != len(data_all)-1:
        

In [ ]:
# # check for any nulls
# data_ActualDate.isnull().sum()

In [ ]:
# # find null value
# null_val = data_ActualDate[data_ActualDate['Date_new'].isnull()].index.tolist()
# data_ActualDate.iloc[null_val]


In [ ]:
# temp = pd.to_datetime(startdate) + pd.DateOffset(days=18644.5+9)
# print(temp)

In [ ]:
# # handle null value by filling in

# data_ActualDate.iloc[1823,2] = pd.to_datetime('12/28/1995')
# data_ActualDate['Date_new'] = pd.to_datetime(data_ActualDate['Date_new']).dt.date

# #data_time = data_time[['Date_new', 'Variable']]
# #data_time.columns = ['Date', 'Variable']

In [ ]:
# data_all = data_time
# data_all.insert(1, 'Variable', list_of_vars)
# data_all.insert(2,'Date_new',data_ActualDate)